In [50]:
#依存構造解析結果から、各トークンと特徴を取得する関数
#モデルのロードは関数外で実施すること
#import spacy
#nlp = spacy.load('ja_ginza')
#get_features(nlp, "テスト用の文章")
def get_features(my_nlp, input_str):
    doc = my_nlp(input_str)

  ###依存構文解析結果の表形式表示
    result_list = []
    for sent in doc.sents:
    #各文を解析して結果をlistに入れる（文章が複数ある場合でもまとめて一つにしてしまう）
        for token in sent:
            try:
                nbor = token.nbor
            except IndexError:
                nbor = None
            info_dict = {}
            features = "i_{0}_pos_{1}_dep_{2}_right_{3}_left_{4}_ent_{5}".format(token.i,token.pos_,token.dep_,''.join([t.text for t in token.rights]),''.join([t.text for t in token.lefts]),token.ent_type_ )
            info_dict["features"] = features
            info_dict["text"] = token.orth_ 
            info_dict["right"] = ''.join([t.text for t in token.rights])
            result_list.append(info_dict)

    return(result_list)

In [62]:
#5w1hを出力するための辞書。keyが5w1hとなる特徴を表し, 値が5w1hの中のどれにマッチしたかを表す。
dictionary = {"^(?=.*(NOUN|PROPN))(?=.*ent_(Country|Province))":"WHERE","^(?=.*(VERB|ADJ))(?=.*ので)":"Why", "^(?=.*(VERB|ADJ))(?=.*(ROOT|amod|acl))":"HOW","^(?=.*NOUN)(?=.*nmod)":"HOW",
              "^(?=.*NOUN)(?=.*(nsubj|iobj))":"WHO","^(?=.*NOUN)(?=.*right_(に|を))(?=.*obl)":"WHAT","^(?=.*NOUN)(?=.*compound)":"WHAT",}

In [64]:
import re
import spacy
nlp = spacy.load('ja_ginza')
#   #接頭/接尾などが加わった形で出力
merge_nps = nlp.create_pipe("merge_noun_chunks")
nlp.add_pipe(merge_nps)
target_str = "私は大阪いたので簡単です"
F = get_features(nlp, target_str)
for f in F:
    print(f["features"])
    for d in dictionary:

        if re.search(d,f["features"]):
            print("{0}:{1}".format(dictionary[d],f["text"]+f["right"]))  
            break

i_0_pos_PRON_dep_iobj_right_は_ent_
i_1_pos_ADP_dep_case_right__ent_
i_2_pos_PROPN_dep_iobj_right__ent_Province
WHERE:大阪
i_3_pos_VERB_dep_advcl_right_たので_ent_
Why:いたので
i_4_pos_AUX_dep_aux_right__ent_
i_5_pos_SCONJ_dep_mark_right__ent_
i_6_pos_AUX_dep_aux_right__ent_
i_7_pos_ADJ_dep_ROOT_right_です_ent_
HOW:簡単です
i_8_pos_AUX_dep_aux_right__ent_
